# Prototype of a Generic Workflow

## 1. Setup Computational Environment



In [1]:
# These would be needed for the tool, but may not all be used here
import xarray as xr
import xesmf as xe
import numpy as np
import glob
import sys
import yaml

# This is only needed in this prototype to verify results
import matplotlib.pyplot as plt

example_config_file = './prototype_config.yml' # < should be input to main()


## 2. Open `config.yml` File

### 2a. Get Job Tag
The user must specify a single alpha numeric string that will be used to label the regridder NetCDF file

### 2b. Are We Building Weights or Just Regridding with Existing Weights
Need build_weights == True/False and
True -> Will need to call regridder
False -> User specifies regridder file name

Regridding == True/False
True -> Need start/end date, variables to regrid, source grid location, destination grid location
False -> Don't need anything... just saving the 

If False & False... throw error... why is the user doing this? 

`TODO` Add a check to make sure key blocks exist in yaml config file

In [2]:
with open(example_config_file, 'r') as config_file:
    config_data = yaml.load(config_file, Loader=yaml.FullLoader)
    

In [3]:
config_data['jobinfo']['jobtag']

'livneh_to_id-wrf30-narr'

In [4]:
if(config_data['jobinfo']['build_weights'] == True):
    print('Yes, we are building weight matrix...')
else:
    print('No, we are not building weight matrix')

Yes, we are building weight matrix...


In [5]:
if((config_data['jobinfo']['build_weights'] == False) and 
   (config_data['jobinfo']['regrid_data'] == False)):
    print('It\'s not clear we\'re doing anything...')
else:
    print('We\'re doing something...')

We're doing something...


## 3. Building Weights (`build_weights==True`)

### 3a. Get Name of Source Grid Metadata File

In [6]:
src_gridinfo_fname = config_data['srcinfo']['src_gridinfo']
print(src_gridinfo_fname)

../debug_grid_metadata/livneh_parms.yml


### 3b. Get Name of Destination Grid Metadata File

In [7]:
dest_gridinfo_fname = config_data['destinfo']['dest_gridinfo']
print(dest_gridinfo_fname)

../dest_grid_metadata/id-wrf30-narr.yml


### 3c. Open Source Grid Example File

Is it curvilinear or rectilinear (`curvilinear`==True/False in metadata.yml file)


In [8]:
with open(src_gridinfo_fname, 'r') as yaml_file:
    src_gridinfo = yaml.load(yaml_file, Loader=yaml.FullLoader)

print('Is source grid curvilinear? Survey says... '+str(src_gridinfo['curvilinear']))
    
# Example error checks
assert src_gridinfo['latvar_name'], 'Source latvar_name key does not exist in'+src_gridinfo_fname
assert src_gridinfo['lonvar_name'], 'Source latvar_name key does not exist in'+src_gridinfo_fname

if(src_gridinfo['curvilinear']):
    assert (src_gridinfo['latvar_b_name'] is not None), 'For curvilinear grids, name of the variable containing cell borders is required'
    assert (src_gridinfo['lonvar_b_name'] is not None), 'For curvilinear grids, name of the variable containing cell borders is required'
    
assert (src_gridinfo['template_file'] is not None), 'Name of a template file must be specified in'+src_gridinfo_fname

Is source grid curvilinear? Survey says... False


Error check:
- Do the latitude and longitude variables exist?
- If it is curvilinear are variable names for corners of cells provided? 
Check to see if lat and long are 1- or 2-D 

If 1-D: (by definition this is rectilinear... so error trap here?) 
- Create corner array
- Use meshgrid to make 2-D

If 2-D: 
- Create corner arrays (only works if rectilinear) 


In [9]:
src_ds = xr.open_dataset(src_gridinfo['template_file'])
print(src_ds)

<xarray.Dataset>
Dimensions:  (lon: 928, lat: 614, time: 31)
Coordinates:
  * lon      (lon) float64 -125.0 -124.9 -124.8 -124.8 ... -67.16 -67.09 -67.03
  * lat      (lat) float64 14.66 14.72 14.78 14.84 ... 52.78 52.84 52.91 52.97
  * time     (time) datetime64[ns] 1986-10-01 1986-10-02 ... 1986-10-31
Data variables:
    Prec     (time, lat, lon) float32 ...
    Tmax     (time, lat, lon) float32 ...
    Tmin     (time, lat, lon) float32 ...
    wind     (time, lat, lon) float32 ...
Attributes:
    CDI:                       Climate Data Interface version 1.6.4 (http://c...
    Conventions:               CF-1.4
    history:                   Sat Oct 11 08:19:20 2014: cdo ifthenelse /Volu...
    nco_openmp_thread_number:  1
    NCO:                       4.4.5
    CDO:                       Climate Data Operators version 1.6.4 (http://c...


### 3d. Open Destination Grid Example File
Is it curvilinear or rectilinear (`curvilinear`==True/False in metadata.yml file)

Error check:
- Do the latitude and longitude variables exist?
- If it is curvilinear are variable names for corners of cells provided? 

Check to see if lat and long are 1- or 2-D 

If 1-D: (by definition this is rectilinear... so error trap here?) 
- Create corner array
- Use meshgrid to make 2-D

If 2-D: 
- Create corner arrays (only works if rectilinear) 

In [10]:
with open(dest_gridinfo_fname, 'r') as yaml_file:
    dest_gridinfo = yaml.load(yaml_file, Loader=yaml.FullLoader)

print('Is source grid curvilinear? Survey says... '+str(dest_gridinfo['curvilinear']))
    
# Example error checks
assert dest_gridinfo['latvar_name'], 'Source latvar_name key does not exist in'+dest_gridinfo_fname
assert dest_gridinfo['lonvar_name'], 'Source latvar_name key does not exist in'+dest_gridinfo_fname

if(dest_gridinfo['curvilinear']):
    assert (dest_gridinfo['latvar_b_name'] is not None), 'For curvilinear grids, name of the variable containing cell borders is required'
    assert (dest_gridinfo['lonvar_b_name'] is not None), 'For curvilinear grids, name of the variable containing cell borders is required'
    
assert (dest_gridinfo['template_file'] is not None), 'Name of a template file must be specified in'+dest_gridinfo_fname

Is source grid curvilinear? Survey says... True


In [11]:
dest_ds = xr.open_dataset(dest_gridinfo['template_file'])
print(dest_ds)

<xarray.Dataset>
Dimensions:     (Time: 1, south_north: 289, west_east: 339, south_north_stag: 290, west_east_stag: 340, land_cat: 21, soil_cat: 16, month: 12, num_urb_params: 132)
Dimensions without coordinates: Time, south_north, west_east, south_north_stag, west_east_stag, land_cat, soil_cat, month, num_urb_params
Data variables: (12/57)
    Times       (Time) |S19 ...
    XLAT_M      (Time, south_north, west_east) float32 ...
    XLONG_M     (Time, south_north, west_east) float32 ...
    XLAT_V      (Time, south_north_stag, west_east) float32 ...
    XLONG_V     (Time, south_north_stag, west_east) float32 ...
    XLAT_U      (Time, south_north, west_east_stag) float32 ...
    ...          ...
    OL4         (Time, south_north, west_east) float32 ...
    VAR_SSO     (Time, south_north, west_east) float32 ...
    LAKE_DEPTH  (Time, south_north, west_east) float32 ...
    URB_PARAM   (Time, num_urb_params, south_north, west_east) float32 ...
    IMPERV      (Time, south_north, west_e

### 3e. Prepare the Source Grid for `xESMF.Regridder()`  

In [12]:
def prep_grid(gridinfo_dict):

    iscurvilinear = gridinfo_dict['curvilinear']
    
    latvar_name = gridinfo_dict['latvar_name']
    lonvar_name = gridinfo_dict['lonvar_name']

    if iscurvilinear:
        latvar_b_name = gridinfo_dict['latvar_b_name']
        lonvar_b_name = gridinfo_dict['lonvar_b_name']
    else: # See if the user entered variable names for latvar_b_name and lonvar_b_name, even if 
        # The grid is rectilinear
            
        if(('latvar_b_name' in gridinfo_dict) and 
           ('lonvar_b_name' in gridinfo_dict)):
            latvar_b_name = gridinfo_dict['latvar_b_name']
            lonvar_b_name = gridinfo_dict['lonvar_b_name']
        else:
            latvar_b_name = None
            lonvar_b_name = None
            
    grid_template_fname = gridinfo_dict['template_file']
    
    ds = xr.open_dataset(grid_template_fname)
    
    lat = ds[latvar_name].data
    lon = ds[lonvar_name].data
    
    if(ds[latvar_name].ndim==1):
        lat = ds[latvar_name].data
    elif(ds[latvar_name].ndim==2):
        lat = ds[latvar_name].data    
    else:
        dropdim = gridinfo_dict['latvar_dropdim']
        lat = ds[latvar_name].isel({dropdim: 0}).data

    if(ds[lonvar_name].ndim==1):
        lon = ds[lonvar_name].data
    elif(ds[latvar_name].ndim==2):
        lon = ds[lonvar_name].data    
    else:
        dropdim = gridinfo_dict['lonvar_dropdim']
        lon = ds[lonvar_name].isel({dropdim: 0}).data
            
    # Boundaries of cells
    if(iscurvilinear): # If curvilinear, these must be provided
        # Get boundary latitude
        if(ds[latvar_b_name].ndim==2):
            lat_b = ds[latvar_b_name].data
        else:
            dropdim = gridinfo_dict['latvar_b_dropdim']
            lat_b = ds[latvar_b_name].isel({dropdim: 0}).data

        # Get boundary latitude
        if(ds[lonvar_b_name].ndim==2):
            lon_b = ds[lonvar_b_name].data
        else:
            dropdim = gridinfo_dict['lonvar_b_dropdim']
            lon_b = ds[lonvar_b_name].isel({dropdim: 0}).data

    # If rectilinear, these *might* have been provided
    elif (iscurvilinear == False) and (latvar_b_name is not None) and (lonvar_b_name is not None): 
        # Get boundary latitude
        if(ds[latvar_b_name].ndim==2):
            lat_b = ds[latvar_b_name].data
        else:
            dropdim = gridinfo_dict['latvar_b_dropdim']
            lat_b = ds[latvar_b_name].isel({dropdim: 0}).data

        # Get boundary latitude
        if(ds[lonvar_b_name].ndim==2):
            lon_b = ds[lonvar_b_name].data
        else:
            dropdim = gridinfo_dict['lonvar_b_dropdim']
            lon_b = ds[lonvar_b_name].isel({dropdim: 0}).data
    
    else: # Calculate corners for rectilinear cases where boundaries not provided
        
        if(lat.ndim==1) and (lon.ndim==1):
            lat_b = np.append((lat[0]-0.5*(lat[1]-lat[0])),(lat+0.5*(lat[1]-lat[0])))
            lon_b = np.append((lon[0]-0.5*(lon[1]-lon[0])),(lon+0.5*(lon[1]-lon[0])))
        
        else:
            lat_b = np.concatenate(((np.expand_dims(lat[0,:]-0.5*(lat[1,:]-lat[0,:]),axis=0)),
                                    np.array(lat+0.5*(lat[1,:]-lat[0,:]))),axis=0)
            lat_b = np.concatenate((lat_b,np.expand_dims(lat_b[:,-1],axis=1)),axis=1)
            
            lon_b = np.concatenate((np.expand_dims(lon[:,0]-0.5*(lon[:,1]-lon[:,0]),axis=1),
                                    (lon+0.5*(lon[:,1]-lon[:,0]))),axis=1)
            lon_b = np.concatenate((lon_b,np.expand_dims(lon_b[-1,:],axis=0)),axis=0)
            
    # If lat, lat_b, lon, and lon_b are 1-D arrays, use meshgrid to make 2-D arrays
    if(lat.ndim==1) and (lon.ndim==1):
        lon, lat = np.meshgrid(lon,lat)
        
    if(lat_b.ndim==1) and (lon_b.ndim==1):
        lon_b, lat_b = np.meshgrid(lon_b,lat_b)

    grid_for_esmf = {'lon': lon, 'lat': lat,
            'lon_b': lon_b, 'lat_b': lat_b}

    return grid_for_esmf


In [13]:
src_grid = prep_grid(src_gridinfo)
print(src_grid)

{'lon': array([[-124.96875, -124.90625, -124.84375, ...,  -67.15625,  -67.09375,
         -67.03125],
       [-124.96875, -124.90625, -124.84375, ...,  -67.15625,  -67.09375,
         -67.03125],
       [-124.96875, -124.90625, -124.84375, ...,  -67.15625,  -67.09375,
         -67.03125],
       ...,
       [-124.96875, -124.90625, -124.84375, ...,  -67.15625,  -67.09375,
         -67.03125],
       [-124.96875, -124.90625, -124.84375, ...,  -67.15625,  -67.09375,
         -67.03125],
       [-124.96875, -124.90625, -124.84375, ...,  -67.15625,  -67.09375,
         -67.03125]]), 'lat': array([[14.65625, 14.65625, 14.65625, ..., 14.65625, 14.65625, 14.65625],
       [14.71875, 14.71875, 14.71875, ..., 14.71875, 14.71875, 14.71875],
       [14.78125, 14.78125, 14.78125, ..., 14.78125, 14.78125, 14.78125],
       ...,
       [52.84375, 52.84375, 52.84375, ..., 52.84375, 52.84375, 52.84375],
       [52.90625, 52.90625, 52.90625, ..., 52.90625, 52.90625, 52.90625],
       [52.96875, 52.9687

### 3f. Prepare Destination Grid for `xESMF.Regridder()`



In [14]:
dest_grid = prep_grid(dest_gridinfo)
print(dest_grid)

{'lon': array([[-120.66577 , -120.6306  , -120.59543 , ..., -108.804565,
        -108.769394, -108.73422 ],
       [-120.668335, -120.63315 , -120.59796 , ..., -108.80203 ,
        -108.766846, -108.73166 ],
       [-120.6709  , -120.6357  , -120.600494, ..., -108.7995  ,
        -108.7643  , -108.729095],
       ...,
       [-121.50107 , -121.46101 , -121.42096 , ..., -107.979034,
        -107.93898 , -107.898926],
       [-121.50441 , -121.464325, -121.424255, ..., -107.97574 ,
        -107.93567 , -107.895584],
       [-121.50775 , -121.46765 , -121.427536, ..., -107.97246 ,
        -107.93234 , -107.89224 ]], dtype=float32), 'lat': array([[40.251755, 40.2537  , 40.25564 , ..., 40.25564 , 40.2537  ,
        40.251755],
       [40.2786  , 40.28055 , 40.282494, ..., 40.282494, 40.28055 ,
        40.2786  ],
       [40.30545 , 40.307404, 40.309345, ..., 40.309345, 40.307404,
        40.30545 ],
       ...,
       [47.94009 , 47.94231 , 47.94452 , ..., 47.94452 , 47.94231 ,
        47.9

### 3g. Call `xESMF.Regridder()`


In [15]:
## Way 1: Compute weights
regridder = xe.Regridder(src_grid, dest_grid, config_data['jobinfo']['xesmf_method'])

## Way 2: Use old weights
#regridder = xe.Regridder(src_grid, dest_grid, config_data['jobinfo']['xesmf_method'],
#                          weights='livneh_to_id-wrf30-narr_conservative_614x928_289x339.nc')

regridder

xESMF Regridder 
Regridding algorithm:       conservative 
Weight filename:            conservative_614x928_289x339.nc 
Reuse pre-computed weights? False 
Input grid shape:           (614, 928) 
Output grid shape:          (289, 339) 
Periodic in longitude?      False

In [16]:
new_weight_fname = config_data['jobinfo']['jobtag']+'_'+regridder.filename
new_weight_fname

'livneh_to_id-wrf30-narr_conservative_614x928_289x339.nc'

In [17]:
regridder.to_netcdf(new_weight_fname)

'livneh_to_id-wrf30-narr_conservative_614x928_289x339.nc'

## 4. Regridding

### 4a. Get Start and End Date of Regridding

### 4b. Get Source Grid Location
Error trap... exists? 
Use xr.open_mfdataset()? 

### 4c. Get Variable Names to be Regridded

### 4d. Get Destination Grid Location

